<a href="https://colab.research.google.com/github/n8mcdunna/DS-Unit-2-Linear-Models/blob/master/module2-regression-2/Nathan_LS_DS_212_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [ ]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [ ]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [ ]:
print(df.shape)
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
# Creating feature for beds and baths
df['beds_and_baths'] = df['bedrooms'] + df['bathrooms']
df.head()

bathrooms  bedrooms  ... common_outdoor_space beds_and_baths
0        1.5         3  ...                    0            4.5
1        1.0         2  ...                    0            3.0
2        1.0         1  ...                    0            2.0
3        1.0         1  ...                    0            2.0
4        1.0         4  ...                    0            5.0

[5 rows x 35 columns]

In [ ]:
df.columns

In [ ]:
df['longitude'].describe()

In [ ]:
df['latitude'].describe()

In [ ]:
df['neighborhood'] = pd.cut(df['latitude'], bins= 10)

In [ ]:
list = []

for row in df['neighborhood']:
  if row not in list:
    list.append(row)

In [ ]:
list

In [ ]:
df['amenities'] = df.iloc[:, 9:-2].sum(axis = 1)

In [ ]:
y = df['price']
df.drop('price', axis= 1, inplace= True)
numerical_columns = df.describe().columns
X = df[numerical_columns]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
print('model coefficeints: ', model.coef_)
print('model intercept:    ', model.intercept_)

model coefficeints:  [-5.74218888e+14 -5.74218888e+14  1.79477710e+03 -1.36863324e+04
  3.10910965e+13  3.10910965e+13  3.10910965e+13  3.10910965e+13
  3.10910965e+13  3.10910965e+13  3.10910965e+13  3.10910965e+13
  3.10910965e+13  3.10910965e+13  3.10910965e+13  3.10910965e+13
  3.10910965e+13  3.10910965e+13  3.10910965e+13  3.10910965e+13
  3.10910965e+13  3.10910965e+13  3.10910965e+13  3.10910965e+13
  3.10910965e+13  3.10910965e+13  3.10910965e+13 -4.05105133e+01
  5.74218888e+14 -3.10910965e+13]
model intercept:     -1085092.1349455097


## Metrics

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
print('training MAE:  ', mean_absolute_error(y_train, model.predict(X_train)))
print('testing MAE:   ', mean_absolute_error(y_test, model.predict(X_test)))
print('training RMSE: ', mean_squared_error(y_train, model.predict(X_train), squared= False))
print('testing RMSE:  ', mean_squared_error(y_test, model.predict(X_test), squared= False))
print('training R^2:  ', model.score(X_train, y_train))
print('testing R^2:   ', model.score(X_test, y_test))

training MAE:   699.9891349037644
testing MAE:    691.468825682055
training RMSE:  1087.524172880993
testing RMSE:   1080.483641652176
training R^2:   0.6218865996707121
testing R^2:    0.6160515086309668


In [ ]:
# Metrics for features: 'beds_and_baths', 'latitude', 'longitude'
training MAE:   815.9239854624547
testing MAE:    809.1590350019839
training RMSE:  1252.3229778384643
testing RMSE:   1229.7699926253758
training R^2:   0.5007630869099182
testing R^2:    0.4959125546705961

In [ ]:
# Metrics for features: 'beds_and_baths', 'latitude', 'longitude', 'amenities'
training MAE:   791.0323851282159
testing MAE:    800.8054712585572
training RMSE:  1216.724372630212
testing RMSE:   1254.2699014039158
training R^2:   0.5170661719961316
testing R^2:    0.5126385698943776

In [ ]:
# Metrics for all numerical features
training MAE:   699.9891349037644
testing MAE:    691.468825682055
training RMSE:  1087.524172880993
testing RMSE:   1080.483641652176
training R^2:   0.6218865996707121
testing R^2:    0.6160515086309668